In [1]:
#!pip install -U spacy
#!python -m spacy download es_core_news_sm
##!python -m spacy validate
##!python -m spacy link es_core_news_sm en --force
#nltk.download('stopwords')
#!pip install gensim 

In [1]:
#librerias generales
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from os import listdir
import statsmodels.tsa.stattools as stt
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import re
import warnings
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

#librerias para los modelos de series de tiempo
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, DynamicVAR

#Libresrias de pre-procesamiento de texto
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
spanish_stopwords = nltk.corpus.stopwords.words('spanish')
spanish_stopwords.extend(['Junta','Directiva','semestre','bien','entornar','aunque','respictavemente', 'probable',
                         'menos', 'me', '3', 'millón', 'punto', 'si', 'futuro', 'mostrar', 'septiembre', 'enero','1', 'b'
                         'febrero', 'marzo', 'abril', 'mayo', 'junio','julio','agosto', 'octubre', 'noviembre','diciembre'
                          
                         'anterior', 'frente', 'largar', 'tiempo', 'partir', 'cercar', 'señalar', 'mes', 'registrar'
                         'mesar', 'atrás', 'estimar', 'proyectar','miembro', 'europa', 'pb', 'finalmente', 'observar'
                         'registrar', 'segundar', 'sobrar', 'rango', 'ingresar' ])
import scipy as sc
##nlp = sc.load('es_core_news_sm')
import es_core_news_sm
nlp = es_core_news_sm.load()

#Librerias para modelo LDA
import gensim
from gensim import corpora, models

## Cargar series de tiempo en dataframe

In [2]:
data = pd.read_excel(r'C:\Users\usuario\Google Drive\Varios Ivan\1 Maestria\Universidad de los Andes\ProyectoGrado\Desarrollo\dataIndicadores.xlsx', sheet_name = 'Var_Men', encoding='utf-8')

data.Fecha = pd.to_datetime(data.Fecha)
data = data.set_index('Fecha')
data = data.dropna()

cycle, trend = sm.tsa.filters.hpfilter(data.IPC, 1600)
data['IPC'] = cycle

cycle, trend = sm.tsa.filters.hpfilter(data.LIBOR, 1600)
data['LIBOR'] = cycle

for i in data.columns.tolist():
    result = stt.adfuller(data[str(i)])
    pvalue = float(result[1])
    isStationary = pvalue <= 0.05
    print(i)
    print("P-VALUE TASA: ", pvalue, isStationary)
    print('*'*40)
    print(' ')

data_series = data
data_series.tail()

IPP
P-VALUE TASA:  2.1849371506292563e-14 True
****************************************
 
IPC
P-VALUE TASA:  0.005648190130457528 True
****************************************
 
BRENT
P-VALUE TASA:  3.32786817065977e-08 True
****************************************
 
TRM
P-VALUE TASA:  7.805727795377145e-12 True
****************************************
 
ISE
P-VALUE TASA:  2.280294410753305e-10 True
****************************************
 
TASA_POL_MON
P-VALUE TASA:  0.009295971952215324 True
****************************************
 
LIBOR
P-VALUE TASA:  1.5802808923633593e-09 True
****************************************
 


,IPP,IPC,BRENT,TRM,ISE,TASA_POL_MON,LIBOR
Fecha,,,,,,,
2019-01-01,0.005884,0.002757,0.150372,-0.015742,0.010035,0.0,0.000153
2019-02-01,0.009343,0.002397,0.066893,-0.014789,0.000294,0.0,-0.000483
2019-03-01,0.006661,0.000854,0.035741,0.003271,-0.010474,0.0,-0.000319
2019-04-01,0.014265,0.001299,0.064483,0.009561,0.019215,0.0,-0.000310
2019-05-01,0.015928,-0.000613,-0.114148,0.049211,0.000909,0.0,-0.000513


## Cargar documentos en dataframe

In [3]:
#Para listar los archivos de directorios
mypath = "C:/Users/usuario/Google Drive/Varios Ivan/1 Maestria/Universidad de los Andes/ProyectoGrado/Desarrollo/Comunicados/"
files = listdir(mypath)

#Cargar textos en un vector
docVector = []
for i in range(len(files)):
    file = open(mypath + files[i],"r", encoding="utf8") 
    #print(files[i])
    docVector.append({'text': file.read(), 'Date': files[i][:7]})

#cargar textos en un dataframe
docDataF = pd.DataFrame(docVector)
docDataF.tail()

,Date,text
140,2019-02,﻿En su sesión de hoy la Junta Directiva decidi...
141,2019-03,﻿En su sesión de hoy la Junta Directiva decidi...
142,2019-04,﻿En su sesión de hoy la Junta Directiva decidi...
143,2019-05,﻿En su sesión de hoy la Junta Directiva decidi...
144,desktop,[.ShellClassInfo]\nInfoTip=Esta carpeta se ha ...


### Función de limpieza y preprocesamiento de textos

In [4]:
def preprocess(text, remove_stop_words=True):
    wordnet_lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer('spanish')
    document = text
    
    # Remove all the special characters
    document = re.sub(r'\W', ' ', document)

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    #Removing punctuation
    document = re.sub(r'[^\w\s]', '', document)
    
    #Removing numbers
    document = ''.join(i for i in document if not i.isdigit())

    #LowerCase    
    document = document.lower()
    
    #Split document word a word
    #words_document = document.split()
    
    words_document = []    
    doc = nlp(document)
    for token in doc: words_document.append(token.lemma_)

    #Remove stop words
    if remove_stop_words:
        words_document = [word for word in words_document if word not in spanish_stopwords]
    
    #Lemmatisation
    #words_document = [wordnet_lemmatizer.lemmatize(word) for word in words_document]
    #words_document = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words_document]
    
    #stimming
    #words_document = [stemmer.stem(word) for word in words_document] 
           
    #return ' '.join(words_document)
    return words_document

### Función de vectorización

In [5]:
def vectorizarTextos (documents_):
    dictionary = gensim.corpora.Dictionary(documents_)
    #Filter out tokens that appear in too much documents
    #less than 15 documents (absolute number) or
    #more than 0.5 documents (fraction of total corpus size, not absolute number).
    #after the above two steps, keep only the first 100000 most frequent tokens.
    dictionary.filter_extremes(no_below=20, no_above=0.9, keep_n=10000)
    #print(dictionary.token2id)
    #len(dictionary)
    bow_corpus = [dictionary.doc2bow(doc) for doc in documents_]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    
    return dictionary, corpus_tfidf

### Función para hallar los parámetros del modelo LDA

In [6]:
## prueba paralelizar
import multiprocessing as mp


def fitLDA_Model(corpus_tfidf_ 
                 ,dictionary_ 
                 ,rangAlpha = np.arange(0.2, 0.6, 0.05)
                 ,rangEta = np.arange(0.4, 3, 0.4)
                 ,rangTopics = range(3, 6, 1)
                 ,resNumTopPala = 30):
    
    rangAlpha = rangAlpha.round(decimals=2)
    rangEta = rangEta.round(decimals=2)
    
    # Step 1: Init multiprocessing.Pool()
    pool = mp.Pool(mp.cpu_count())
    # Step 2: call paralell
    resultMatrix_ = [pool.apply(fitLDA_ModelParallel, args=(corpus_tfidf_, dictionary_, alpha, rangEta, rangTopics, resNumTopPala)) for alpha in rangAlpha]
    pool.close() 
    return resultMatrix_



def fitLDA_ModelParallel(corpus_tfidf_, dictionary_, alpha_, rangEta, rangTopics, resNumTopPala):
    
    resultMatrixParalell_ = []
    for eta_ in rangEta:
        for num_topics_ in rangTopics:
            print(alpha_, eta_, num_topics_ )
            #Ajustar el modelo con la configuración que corresponda en el for
            lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf_, 
                                                         id2word = dictionary_, 
                                                         alpha = alpha_, 
                                                         eta = eta_, 
                                                         num_topics = num_topics_, 
                                                         workers=-1,
                                                         random_state=42)

            #Obtener la distribución de los documentos en los tópicos
            topics = lda_model_tfidf.get_document_topics(corpus_tfidf_, 
                                                         minimum_probability=None, 
                                                         minimum_phi_value=None, 
                                                         per_word_topics=False)

            #Guarda el top n paralabra en el resultado, según parámetro resNumTopPala
            top_Palabras = []
            for i in range(0,num_topics_):
                top_Palabras.extend( np.append(np.full((resNumTopPala,1), i+1), lda_model_tfidf.show_topic(i, topn=resNumTopPala), axis=1 ) )

            resultMatrixParalell_.append([alpha_, eta_, num_topics_, topics, top_Palabras])
    
    
    return resultMatrixParalell_

### Función para calcular modelos VAR y LDA

In [7]:
def fit_VARModel(docDataF_, indicadorSerie, data_series_, num_periodos_test):

    resultSeries_ = []

    #correr el ajuste del LDA mes a mes por el numero de periodos de test seleccionados
    #en cada iteración se calcula un gridsearch con los parametros del LDA
    for i in range (0, num_periodos_test):
        
        documents=[]
        #1.1 Correr función "preprocess" enviando el texto mensualizado removiendo los periodos de test 1 a la vez 
        for texto in docDataF_[:-num_periodos_test+i].text:
            documents.append(preprocess(texto))
        print("num_mes_test:", i, "num_docs", len(documents) )
        
        #1.2 Vectorizar los textos con TFIDF, la funcion retorna el diccionario y el texto vectorizado
        dictionary, corpus_tfidf = vectorizarTextos (documents)

        #1.3 Ajustar los modelos LDA con las posible combinaciones de alpha, eta y número de topicos
        # Retorna la distribución de tópicos por cada documento de acuerdo con el modelo LDA ajustado con los diferentes parámetros
        paramsTopics_ = fitLDA_Model(corpus_tfidf, dictionary)
        
        result_ = []
        #Correr un modelo VAR para cada resultado de LDA de acuerdo con el gridsearch
        for num_model in range(0,len(paramsTopics_)):
            topics = paramsTopics_[num_model][3]
            #Crear un Dataframe de topicos en los documentos como serie de tiempo
            topics_df = pd.DataFrame(gensim.matutils.corpus2csc(topics).T.toarray())
            
            #Unir los topicos con el indicador en el mismo DataFrame
            data = indicadorSerie[:-num_periodos_test+i].reset_index().iloc[:,1:].join(topics_df).join(data_series_[:-num_periodos_test+i])
            #data.to_pickle("Data.pkl") #Guardar resultado parcial
            
            #El numero 1 indica el numero de resagos
            model = VAR(data)            
            results = model.fit(1)

            #proyectar test
            proy_var = pd.DataFrame(results.forecast(data.values[-results.k_ar:], 1))
            #print("num_model:", num_model, " proy:", proy_var.values[:,0])
            
            result_.append([ proy_var.values[:,0], indicadorSerie.iloc[-num_periodos_test+i,0] ] )
            
        #print(result_)
        
        resultSeries_.append(np.append(result_, paramsTopics_, axis=1) )        
    
    return resultSeries_

# Flujo de ejecución

In [ ]:
print("Inicio =", datetime.now())
otras_series = data_series.reset_index().drop(["Fecha", "TASA_POL_MON"], axis=1)
resultSeries = fit_VARModel(docDataF, data_series[["TASA_POL_MON"]],otras_series, 24)
print("Fin =", datetime.now())

# Ultima ejecución
# Inicio = 2019-08-31 16:07:48.445866
# Fin = 2019-08-31 23:59:49.450381

Inicio = 2019-09-17 22:54:49.540486
num_mes_test: 0 num_docs 121


In [ ]:
## Guardar resultado
fileObject = open("Result_17Sep_par.pkl", 'wb')
pickle.dump(resultSeries, fileObject)
fileObject.close()

# Carga y transforma resultados para reporte

In [14]:
## Cargar resultado guardado
fileObject2 = open("Result_16Sep.pkl", 'rb')
resultLoaded = pickle.load(fileObject2)
fileObject2.close()
# Leer un registro para validar
resultLoaded[1][1][5]

## Transformar y crear arrays

In [15]:
#Nivel 1: Mes BackTest
#Nivel 2: Modelo LDA
#Nivel 3: Proyeccion, valor Original, Parámetros, tópicos y palabras
#resultLoaded[0][0][:4]
dataModel = []
modelTopicos = []
modelPalabras = []
for nivel1 in range(0,len(resultLoaded)): 
    for nivel2 in range(0,len(resultLoaded[nivel1])): 
        
        dataModel.append( np.append([nivel1],
                           np.append([nivel2], 
                           resultLoaded[nivel1][nivel2][:5], axis=0), axis=0))
        
        lonTopicos = len(resultLoaded[nivel1][nivel2][5])
        modelTopicos.extend(np.append(np.full((lonTopicos,1), nivel1),
                            np.append(np.full((lonTopicos,1), nivel2),
                            np.append(np.indices((lonTopicos,1))[0] ,
                            gensim.matutils.corpus2csc(resultLoaded[nivel1][nivel2][5]).T.toarray(), axis=1), axis=1), axis=1))
        
        lonPalabra = len(resultLoaded[nivel1][nivel2][6])
        modelPalabras.extend(np.append(np.full((lonPalabra,1), nivel1),
                             np.append(np.full((lonPalabra,1), nivel2),
                             resultLoaded[nivel1][nivel2][6], axis=1), axis=1))

## Crear dataframes y transformar formatos

In [16]:
#import datetime
#import calendar

#Crea Dataframe modelos
dataModel_df = pd.DataFrame(dataModel, columns=['MesBack','ModeloLDA','Proyec','OrgValue','Alpha','Eta','Topics' ])
fec_end = max(data_series.index)
lenback = max(dataModel_df.MesBack)
dataModel_df.MesBack = dataModel_df.MesBack.apply(lambda x: fec_end - relativedelta(months=lenback-x))

#Crea Dataframe topicos
modelTopicos_df = pd.DataFrame(modelTopicos)
modelTopicos_df.rename(columns= {0: "MesBack", 1:"Modelo", 2:"MesDocumento", 3:'T1',4:'T2',5:'T3',6:'T4',7:'T5',8:'T6',9:'T7',10:'T8',11:'T9'}, inplace = True) 
modelTopicos_df.MesBack = modelTopicos_df.MesBack.apply(lambda x: fec_end - relativedelta(months=lenback-x))

#Crea Dataframe palabras
modelPalabras_df = pd.DataFrame(modelPalabras, columns=['MesBack','ModeloLDA','Topic','Palabra','Importancia'])
modelPalabras_df.MesBack = modelPalabras_df.MesBack.apply(lambda x: fec_end - relativedelta(months=lenback-x.astype(int)))
modelTopicos_df.Modelo = modelTopicos_df.Modelo.astype(int)
fec_ini = min(data_series.index)
modelTopicos_df.MesDocumento = modelTopicos_df.MesDocumento.apply(lambda x: fec_ini + relativedelta(months=x))


In [17]:
dataModel_df.head()

,MesBack,ModeloLDA,Proyec,OrgValue,Alpha,Eta,Topics
0,2016-06-01,0,[0.0029156301930015616],0.0,0.4,0.4,4
1,2016-06-01,1,[0.0035906666096323647],0.0,0.4,0.4,5
2,2016-06-01,2,[0.002533123298007922],0.0,0.4,0.4,6
3,2016-06-01,3,[0.002127794713487674],0.0,0.4,0.4,7
4,2016-06-01,4,[0.0021549370540014934],0.0,0.4,0.4,8


In [18]:
modelTopicos_df.head()

,MesBack,Modelo,MesDocumento,T1,T2,T3,T4,T5,T6,T7,T8,T9
0,2016-06-01,0,2007-06-01,0.082921,0.069299,0.769925,0.077856,NaN,NaN,NaN,NaN,NaN
1,2016-06-01,0,2007-07-01,0.071562,0.070857,0.082140,0.775442,NaN,NaN,NaN,NaN,NaN
2,2016-06-01,0,2007-08-01,0.072621,0.075779,0.672315,0.179285,NaN,NaN,NaN,NaN,NaN
3,2016-06-01,0,2007-09-01,0.060361,0.064345,0.068391,0.806903,NaN,NaN,NaN,NaN,NaN
4,2016-06-01,0,2007-10-01,0.078286,0.509768,0.088551,0.323395,NaN,NaN,NaN,NaN,NaN


In [19]:
modelPalabras_df.head()

,MesBack,ModeloLDA,Topic,Palabra,Importancia
0,2016-06-01,0,1,ajustar,0.0075390977
1,2016-06-01,0,1,resultar,0.0070804204
2,2016-06-01,0,1,pesar,0.0069785225
3,2016-06-01,0,1,gastar,0.0062734066
4,2016-06-01,0,1,fenómeno,0.0061773825


## Exportar a csv

In [20]:
dataModel_df.to_csv("Modelos.csv", index = False)
modelTopicos_df.to_csv("Topicos.csv", index = False)
modelPalabras_df.to_csv("Palabras.csv", index = False)

# Codigo de pruebas

In [0]:
#Ejemplo para:
#Nivel 1: Primer mes de Back Test 0
#Nivel 2: Modelo LDA 0
#Nivel 3: Se imprime a continuación
mes= 3
model = 10
print("Error",resultLoaded[mes][model][0], "Alpha: ",resultLoaded[mes][model][1], " Eta:", resultLoaded[mes][model][2], " NumTopics:", resultLoaded[mes][model][3])

#Convertir el array top de palabras en un DataFrame
palabrasModel = pd.DataFrame(resultLoaded[mes][model][5], columns=['Topic','Palabra','Importancia'])
palabrasModel.head()

Error 0.0009258239206246799 Alpha:  0.4  Eta: 2  NumTopics: 6


,Topic,Palabra,Importancia
0,1,2013,0.0033333746
1,1,octubr,0.0029816923
2,1,2012,0.002935449
3,1,agreg,0.0029316684
4,1,9,0.0029021103


In [0]:
######################################
##########       TEST      ###########
######################################

documents=[]

#1.1 Correr función "preprocess" enviand el texto de al cual se le va aplicar LDA, el resultado se guarda en el vector documents
for texto in docDataF.text:    
    documents.append(preprocess(texto))

#1.2 Vectorizar los textos con TFIDF, la funcion retorna el diccionario y el texto vectorizado
dictionary, corpus_tfidf = vectorizarTextos (documents)

num_topics_=8

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             id2word = dictionary, 
                                             alpha = 0.5, 
                                             eta = 1, 
                                             num_topics=num_topics_, 
                                             workers=3, 
                                             random_state=42)
                
#Obtener la distribución de los documentos en los tópicos
topics = lda_model_tfidf.get_document_topics(corpus_tfidf, 
                                             minimum_probability=None, 
                                             minimum_phi_value=None, 
                                             per_word_topics=False)



In [0]:
# Prueba Paralelo
otras_series = data_series.reset_index().drop(["Fecha", "TASA_POL_MON"], axis=1)
resultSeries = fit_VARModel(docDataF, data_series[["TASA_POL_MON"]],otras_series, 2)

### Función para probar todos los inidcadores 
topicsResult = []
for column in data_series.columns.tolist():
    print(column)
    otras_series = data_series.reset_index().drop(["Fecha", column], axis=1)
    resultSeries = fit_VARModel(docDataF, data_series[[column]],otras_series, 36)
    topicsResult.append(resultSeries)

In [0]:
print(gensim.matutils.corpus2csc(topics).T.toarray())

[[0.12428228 0.17196493 0.11789083 ... 0.11286914 0.11657    0.10962423]
 [0.13229154 0.11617595 0.12517989 ... 0.12548698 0.14659594 0.11900239]
 [0.13345174 0.12221934 0.12376492 ... 0.11004317 0.13608006 0.12248868]
 ...
 [0.0994661  0.20076783 0.10465934 ... 0.10093307 0.11317815 0.10028048]
 [0.10671195 0.19951363 0.11116373 ... 0.10870337 0.11943376 0.11263458]
 [0.10673277 0.19945826 0.11119266 ... 0.10870181 0.11931256 0.11272751]]


In [0]:
topics_df = pd.DataFrame(gensim.matutils.corpus2csc(topics).toarray())
topics_df

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,0.125256,0.135151,0.137469,0.120354,0.122634,0.137607,0.122881,0.130394,0.142868,0.114962,...,0.095682,0.095289,0.111257,0.111300,0.104713,0.096974,0.096975,0.099460,0.106783,0.106803
1,0.171536,0.115306,0.121951,0.112687,0.127061,0.108558,0.124716,0.116441,0.121370,0.127640,...,0.246504,0.258952,0.146898,0.145404,0.193004,0.212204,0.211787,0.199862,0.199634,0.199804
2,0.117550,0.124519,0.121891,0.136881,0.125538,0.116589,0.119392,0.132343,0.125407,0.115773,...,0.094832,0.098993,0.114193,0.114423,0.100805,0.098306,0.098297,0.104670,0.111232,0.111313
3,0.127767,0.120899,0.138075,0.133413,0.125148,0.144588,0.125795,0.125956,0.130481,0.114869,...,0.097646,0.107617,0.101228,0.101265,0.099968,0.098701,0.098690,0.099837,0.102441,0.102490
4,0.118945,0.114608,0.112483,0.120352,0.112777,0.116355,0.123473,0.135826,0.133234,0.119134,...,0.182968,0.146491,0.213900,0.214873,0.207748,0.181649,0.182117,0.181620,0.138999,0.138657
5,0.112994,0.127639,0.109974,0.109293,0.121268,0.116600,0.110925,0.111438,0.109879,0.113718,...,0.093640,0.096468,0.100131,0.100154,0.096800,0.097073,0.097068,0.100961,0.108689,0.108761
6,0.116301,0.143190,0.134926,0.142858,0.129105,0.135886,0.117717,0.126602,0.123553,0.184790,...,0.093233,0.096047,0.103909,0.103955,0.102516,0.118617,0.118586,0.113274,0.119457,0.119392
7,0.109652,0.118689,0.123231,0.124162,0.136469,0.123816,0.155102,0.120999,0.113208,0.109113,...,0.095495,0.100142,0.108483,0.108625,0.094446,0.096477,0.096480,0.100316,0.112766,0.112780
